In [1]:
import traceback
import os
import json
import time
import ml_collections
import lightning as L
import torch
from lightning.pytorch.callbacks import ModelCheckpoint, LearningRateMonitor
from lightning.pytorch import seed_everything
from lightning.pytorch.loggers import WandbLogger
from seisLM.model.foundation.pretrained_models import LitMultiDimWav2Vec2
from seisLM.data_pipeline import collator
from seisLM.data_pipeline import seisbench_dataloaders as dataloaders
from seisLM.utils import project_path
from seisLM.utils.wandb_utils import shutdown_cleanup_thread
from seisLM.utils.config_utils import TrackedConfig

In [ ]:
config_path = '/scicore/home/dokman0000/liu0003/projects/seisLM/seisLM/configs/pretrain/pretrain_config.json'

with open(config_path, "r", encoding="utf-8") as f:
  config = json.load(f)
config = ml_collections.ConfigDict(config)
config = TrackedConfig(config)

DEFAULT_NUM_WORKERS = 8

print("Running in test mode")
config.training_config.max_epochs = 1
config.data_config.data_name = ['ETHZ']
config.data_config.local_batch_size = 8
project_name = "test_pretrained_seisLM"



Running in test mode


In [ ]:
config.get_accessed_keys()

{'data_config', 'training_config'}

In [6]:
from ml_collections import config_dict

# class TrackedConfig:
#     def __init__(self, config):
#         self.config = config
#         self.accessed_keys = set()

#     def __getattr__(self, key):
#         if key in self.config:
#             self.accessed_keys.add(key)
#             return self.config[key]
#         else:
#             raise AttributeError(f"Config has no attribute '{key}'")

#     def get_accessed_fields(self):
#         return self.accessed_keys

#     def get_unaccessed_fields(self):
#         return set(self.config.keys()) - self.accessed_keys

import copy
from ml_collections import config_dict


class TrackedConfig:
    def __init__(self, config):
        self.config = config
        self.unaccessed = copy.deepcopy(config)

    def __getattr__(self, key):
      self.config.__getattr__(key)
      self.unaccessed.__delattr__(key)
        # if key in self.config:
            # self.accessed_keys.add(key)
            # return self.config[key]
        # else:
            # raise AttributeError(f"Config has no attribute '{key}'")
    # def get_accessed_keys(self):
    #     return self.accessed_keys

    def get_unaccessed_keys(self):
        return self.unaccessed #set(self.config.keys()) - self.accessed_keys



# Example usage
cfg = config_dict.ConfigDict()
cfg.float_field = 12.6
cfg.integer_field = 123
cfg.another_integer_field = 234
cfg.nested = config_dict.ConfigDict()
cfg.nested.string_field = 'tom'
cfg.nested.string_field2 = 'jerry'

tracking_cfg = TrackedConfig(cfg)
print(tracking_cfg.float_field)  # Access float_field
print(tracking_cfg.nested.string_field)  # Access nested.string_field

# the output here should be cfg.integer_field, cfg.another_integer_field, and cfg.nested.string_field2.
print('unaccessed', tracking_cfg.get_unaccessed_keys())


None


AttributeError: 'NoneType' object has no attribute 'string_field'

In [10]:
tracking_cfg

In [7]:
tracking_cfg.nested

AttributeError: "'nested'"